In [1]:
#Import important Library 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [2]:
# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/House Price/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
#Read the Training the Dataset 
data = pd.read_csv('../House Price/input/train.csv', index_col = 'Id')

In [6]:
#Clean Dataframe of columns with missing values 
cols_with_missing = [ col for col in data.columns if data[col].isnull().any()]
data.drop(cols_with_missing,axis = 1, inplace=True)

In [7]:
#Define the predicition target 
y = data.SalePrice

In [9]:
#Define the features 
X = data.drop(['SalePrice'],axis = 1)

In [10]:
# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [11]:
# Select categorical columns with relatively low cardinality
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]

In [12]:
# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

In [13]:
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [14]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

In [16]:
# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# ('ordinal', OrdinalEncoder(handle_unknown='error')),

In [17]:
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

In [18]:
## Model data ##

# Define
my_model = XGBRegressor(n_estimators=5000, learning_rate=0.025)
#                         early_stopping_rounds=5, early_stopping_metric='mae')
my_pipeline = Pipeline(steps=[
                    ('preprocessor', preprocessor),
                    ('model', my_model)])

In [19]:
# Fit
my_pipeline.fit(X_train, y_train,)
#                 eval_set=[(X_valid, y_valid)])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['MSSubClass', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath',
                                                   'HalfBath', 'BedroomAbvG...
                              feature_types=None, gamma=0, gpu_id=-1,
                              grow_policy='depthwise', importance_type=None,
                              interaction_constraints='', learning_rate=0.025,
                              max_bin=256, max_cat_threshold=64,
                              max_cat_to_onehot=4, max_delta_step=0,
                              max_depth=6, max_leaves=0, min_child_weight=1,
                              missing=nan, monotone_constraints='()',
                              n_estimators=5000, n_jobs=0, num_parallel_tree=1,
                              predictor='auto', random_state=0, ...))])

In [20]:
# Predict
preds = my_pipeline.predict(X_valid)

In [21]:
# Evaluate
mae = mean_absolute_error(y_valid, preds)
print('\n'+str(mae))


16266.608438570205


In [22]:
## Create competition submission ##

X_test = pd.read_csv('../House Price/input/test.csv', index_col='Id')

In [23]:
# Preprocess test data
final_X_test = X_test[my_cols].copy()

In [24]:
# Get test predictions
preds_test = my_pipeline.predict(final_X_test)

In [25]:
# Save test predictions to file
output = pd.DataFrame({'Id': final_X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)